In [600]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
df = pd.read_csv('ML_Data/TSLA_vader.csv')


#BTC_neg.csv
#BTC_pos.csv
#BTC_vader.csv

#DJI_neg.csv
#DJI_pos.csv
#DJI_vader.csv

#TSLA_neg.csv
#TSLA_pos.csv
#TSLA_vader.csv

df = df.iloc[50:]
#df.info()

# Visualize data

In [601]:
#df.head()

In [602]:
#df['label'].value_counts().plot(kind='bar')

# Normalize  data

In [603]:

y = df['label']
X = df.drop(['label','date', 'Unnamed: 0'], axis=1)
X = X.values

#scaler = StandardScaler()
#X = scaler.fit_transform(X)

In [604]:
pipeline = Pipeline([
    ('normalizer', StandardScaler()), #Step1 - normalize data
    ('clf', LogisticRegression()) #step2 - classifier 
])
pipeline.steps

[('normalizer', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('clf',
  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='warn',
            n_jobs=None, penalty='l2', random_state=None, solver='warn',
            tol=0.0001, verbose=0, warm_start=False))]

# Split data

In [605]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(228, 24)
(58, 24)
(228,)
(58,)


# Estimate classifiers

In [613]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

clfs = []
#clfs.append(LogisticRegression(solver='lbfgs',C=0.5570175438596491))
clfs.append(SVC())
#C=0.03,gamma=0.001,kernel='linear'
#clfs.append(GaussianNB())
#clfs.append(KNeighborsClassifier())
#clfs.append(RandomForestClassifier(n_estimators=20))

for classifier in clfs:
    pipeline.set_params(clf = classifier)
    scores = cross_validate(pipeline, X_train, y_train, cv=10)
    
    print('---------------------------------')
    print(str(classifier))
    print('-----------------------------------')
    for key, values in scores.items():
            print(key,' mean ', values.mean())
            #print(key,' std ', values.std())'''

---------------------------------
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
-----------------------------------
fit_time  mean  0.005284953117370606
score_time  mean  0.0009465217590332031
test_score  mean  0.5045454545454546
train_score  mean  0.821631541558134


In [607]:
from sklearn.model_selection import GridSearchCV
pipeline.set_params(clf = SVC())
pipeline.steps

[('normalizer', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('clf', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False))]

In [608]:
cv_grid = GridSearchCV(pipeline, param_grid = {
    #'clf__solver' : ['lbfgs','newton-cg'],
    'clf__kernel' : ['linear', 'rbf'],
    #'clf__C' : np.linspace(0.1,1.2,12),
    'clf__C' : np.linspace(0.01,1.0,100),
    'clf__gamma' : [0.001, 0.01, 0.1, 1,'auto_deprecated']
},cv=3)

#solver='lbfgs'

cv_grid.fit(X_train, y_train)



GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('normalizer', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'clf__kernel': ['linear', 'rbf'], 'clf__C': array([0.01, 0.02, ..., 0.99, 1.  ]), 'clf__gamma': [0.001, 0.01, 0.1, 1, 'auto_deprecated']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [609]:
cv_grid.best_params_

{'clf__C': 0.02, 'clf__gamma': 0.001, 'clf__kernel': 'linear'}

In [610]:
cv_grid.best_estimator_

Pipeline(memory=None,
     steps=[('normalizer', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', SVC(C=0.02, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [611]:
cv_grid.best_score_

0.5701754385964912

In [612]:
y_predict = cv_grid.predict(X_test)
accuracy = accuracy_score(y_test,y_predict)
print('Accuracy of the best classifier after CV is %.5f%%' % (accuracy*100))

Accuracy of the best classifier after CV is 39.65517%


# Output

In [25]:
new_test_df = pd.DataFrame({'date':test_df['date'].values, 'label':pred})

In [15]:
new_test_df.to_csv('ML_Data/new_test_df.csv')